# Importing libraries

In [55]:
import numpy as np # support for multi-dimensional arrays and matrices
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
sns.set(style="white", color_codes=True)

import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error #change this for absolute error (???)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

# Input data files are available in the "./input/" directory.

from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection

import tensorflow as tf

dengue_features_test.csv
dengue_features_train.csv
dengue_labels_train.csv



# Import Data

In [56]:
f_train = pd.read_csv("./input/dengue_features_train.csv", parse_dates=True, delimiter=',')
l_train = pd.read_csv("./input/dengue_labels_train.csv", parse_dates=True, delimiter=',')
f_test = pd.read_csv("./input/dengue_features_test.csv", parse_dates=True, delimiter=',')

print(f_train.shape, l_train.shape, f_test.shape)

(1456, 24) (1456, 4) (416, 24)


# Dropping Some Columns

In [57]:
f_train.drop(['week_start_date'], axis=1, inplace=True)
f_test.drop(['week_start_date'], axis=1, inplace=True)

l_train.drop(['year'], axis=1, inplace=True)
l_train.drop(['weekofyear'], axis=1, inplace=True)
l_train.drop(['city'], axis=1, inplace=True)



# Rearrange the cities

In [58]:
f_train = f_train.replace('sj',0)
f_train = f_train.replace('iq',1)

l_train = l_train.replace('sj',0)
l_train = l_train.replace('iq',1)

f_test = f_test.replace('sj',0)
f_test = f_test.replace('iq',1)

f_test.shape
f_train.shape
l_train.shape

(1456, 1)

# Fill NAs

In [59]:
f_train.fillna(method='ffill',inplace=True)
f_test.fillna(method='ffill',inplace=True)
l_train.fillna(method='ffill',inplace=True)

print(f_test.shape)

(416, 23)


# Multilayer 

In [124]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 23 # 1st layer number of features
n_hidden_2 = 23 # 2nd layer number of features
n_input = l_train.shape[1] # MNIST data input (img shape: 28*28)
n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [256, 23])
# x = tf.transpose(
#     x,
#     perm=None,
#     name='transpose'
#)
y = tf.placeholder("float", [256, ])
y = tf.transpose(
    y,
    perm=None,
    name='transpose2'
)

In [125]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [126]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [128]:
# Construct model
weights = tf.transpose(weights)
pred = multilayer_perceptron(x, weights, biases)
pred = tf.transpose(pred)

TypeError: Failed to convert object of type <class 'dict'> to Tensor. Contents: {'h1': <tf.Variable 'Variable_84:0' shape=(1, 23) dtype=float32_ref>, 'h2': <tf.Variable 'Variable_85:0' shape=(23, 23) dtype=float32_ref>, 'out': <tf.Variable 'Variable_86:0' shape=(23, 1) dtype=float32_ref>}. Consider casting elements to a supported type.

In [90]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [91]:
# Initializing the variables
init = tf.global_variables_initializer()

In [92]:
print(l_train[1:2].iloc[0])

total_cases    5
Name: 1, dtype: int64


In [110]:
x = tf.transpose(x)
y = tf.transpose(y)

In [108]:
print(x.shape, y.shape)

(23, ?) (?,)


In [119]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        print('a')
        total_batch = int(1456/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = f_train[i:i+1]
            batch_y = l_train[i:i+1].iloc[0]
            print(x.shape, y.shape)
            print(batch_x.shape, batch_y.shape)
            print(batch_y[0])
            #print(x,y)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y[0]})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

a
(?, 23) (?,)
(1, 23) (1,)
4


ValueError: Cannot feed value of shape () for Tensor 'transpose_32:0', which has shape '(?,)'

# Features

# Normalize

In [ ]:
f_train = (f_train - f_train.mean())/f_train.std()
f_test = (f_test - f_test.mean())/f_test.std()

In [ ]:
   # Second round of gradient boosting
# Cols = train.columns.values.tolist()
# clf = GradientBoostingRegressor(random_state=1729)
# selector = clf.fit(train, target)

# importances = selector.feature_importances_
# fs = SelectFromModel(selector, prefit=True)
# train = fs.transform(train)
# test = fs.transform(test)
# print(train.shape, test.shape)

# selectedCols = train.shape[1]
# sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
# sortedCols = sortedCols[0:selectedCols]

# Split and Train

In [ ]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(f_train, 
                                                                    l_train, test_size=0.3, random_state=101)

In [ ]:
import xgboost as xgb

from sklearn.cross_validation import KFold

In [ ]:
xgbc = xgb.XGBRegressor(n_estimators = 100, # number of boosted trees
                             learning_rate = 0.01, # step size shrinkage used in update to prevent overfitting
                             max_depth = 7, # maximum depth of a tree
                             subsample = 0.7, # subsample ratio of the training set (Stochastic gradient boosting)
                             colsample_bytree = 0.7,
                             silent = 0) # subsample features
    
xgbc.fit(x_train, y_train)

In [ ]:
print('Training:', mean_absolute_error(y_train, xgbc.predict(x_train)))
print('Validation:', mean_absolute_error(y_val, xgbc.predict(x_val)))

# Finalize

In [ ]:
f_test.shape

xgbc.fit(f_train, l_train)
final = xgbc.predict(f_test)
final.shape

final = np.hstack(final)
final[final < 0] = 0
final = final.astype(dtype='int64')

np.savetxt("final15.csv", final, delimiter=",")

final